In [ ]:
# requirements
import string
import re
import pandas as pd
import textblob
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import numpy as np
import keras
from keras.layers import Dense, Conv1D, Flatten, Dropout, Activation
#Functions
def remove_punc(row):
punc = string.punctuation
temp = ''
for word in row['text']:
if word not in punc:
temp+=word
row['text'] = temp
return row['text']
def splitter_remover(row):
temp = row['text']
temp = temp.split()
temp = [words for words in temp if len(words)>1]
row['text'] = temp
return row['text']
def stopWords(row):
temp= [words for words in row['text'] if words not in stop]
row['text'] = temp
return row['text']
def word_stem(row):
temp = [PorterStemmer().stem(i) for i in row['text']]
row['text'] = temp
return row['text']
from nltk.stem import WordNetLemmatizer
def word_lem(row):
temp = [WordNetLemmatizer().lemmatize(i) for i in row['text']]
row['text'] = temp
return row['text']
def removeInt(row):
temp = [word for word in row['text'] if not isinstance(word,int)]
row['text'] = temp
return row['text']
def strcon(row):
temp = [str(item) for item in row['text']]
row['text'] = temp
return row['text']
def cleanup(row):
temp=''
for i in row['text']:
temp+=i+" "
row['text'] = temp
return row['text']
def tester(y_pred, y_true):
recall = sklearn.metrics.recall_score(y_true, y_pred)
precision = sklearn.metrics.precision_score(y_true, y_pred)
f1 = sklearn.metrics.f1_score(y_true, y_pred)
fbeta = sklearn.metrics.fbeta_score(y_true,y_pred, beta=0.5)
accuracy = sklearn.metrics.accuracy_score(y_true,y_pred)
print('Accuracy', accuracy)
print('Recall score', recall)
print('Precision score', precision)
print('f1 score', f1)
print('fbeta score', fbeta)
return [accuracy, recall, precision, f1, fbeta]
# import data
data = pd.read_csv('Data/final_data.csv', index_col=0)
ham = data[data['ham_spam']==0]
spam = data[data['ham_spam']==1]
test_size = [.25,.5,1,2,4]
data = {'model_name': [], 'accuracy': [], "recall": [], 'precision': []
, "f1_score":[],'fbeta_score': [], 'ham_number': [], 'spam_number':[],
'true_neg':[], 'false_pos':[], 'false_neg':[], 'true_pos': []}
df_final = pd.DataFrame(data=data)
for q in test_size:
ham = ham.sample(n =round(len(spam)*q), random_state=1,
replace=True).reset_index(drop=True)
data_cut = ham.append(spam)
data_cut = data_cut.sample(len(data_cut),random_state=1).reset_index(drop=True)
#data cleaning and feature engineering
#lower text
data_cut['text'] = data_cut['text'].str.lower()
#extra whitespace
data_cut['text'] = data_cut['text'].str.replace(r'\s+'," ")
#next lines
data_cut['text'] = data_cut['text'].str.replace(r'\\n'," ")
data_cut['text'] = data_cut['text'].str.replace(r'\n'," ")
data_cut['text'] = data_cut['text'].str.replace(r'\r'," ")
#subject
data_cut['text'] = data_cut['text'].str.replace('subject',"")
#names
data_cut['text'] = data_cut['text'].str.replace('jeremy randolph',"name")
data_cut['text'] = data_cut['text'].str.replace('jeremy',"name")
#phone number
data_cut['text'] =
data_cut['text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',"contact number")
#email addresses
data_cut['text'] = data_cut['text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',"email")
#currency
data_cut['text'] = data_cut['text'].str.replace(r'£|\$',"money")
#hyperlinks
data_cut['text'] =
data_cut['text'].str.replace(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',"links")
#removing numbers
data_cut['text'] = data_cut['text'].str.replace(r'\d+(\.\d+)?'," ")
#remove special chartacters
data_cut['text'] = data_cut['text'].str.replace(r'[^a-zA-Z0-9]+'," ")
#remove punctuation
data_cut['text'] = data_cut.apply(remove_punc,axis=1)
#split and remove single letter words
data_cut['text'] = data_cut.apply(splitter_remover,axis=1)
#stop words
data_cut['text'] = data_cut.apply(stopWords,axis=1)
#stemming to root words
#data_cut['text'] = data_cut.apply(word_stem,axis=1)
#lemmatization
data_cut['text'] = data_cut.apply(word_lem,axis=1)
#remove int
data_cut['text'] = data_cut.apply(removeInt,axis=1)
#make str
data_cut['text'] = data_cut.apply(strcon,axis=1)
#list to str
data_cut['text'] = data_cut.apply(cleanup,axis=1)
#train test split
x_train,x_test,y_train,y_test = train_test_split(data_cut['text'],data_cut['ham_spam'],
random_state=1, test_size=0.1)
#y_train = np.asarray(y_train)
#y_test = np.asarray(y_test)
#tfidf
vector = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
features_train = vector.fit_transform(x_train)
features_test = vector.transform(x_test)
#Naive Bayes
nb = MultinomialNB()
nb.fit(features_train, y_train)
#score_train = nb.score(features_train, y_train)
#score_test = nb.score(features_test, y_test)
y_pred = nb.predict(features_test)
#matrix = confusion_matrix(y_test,y_pred)
#matrix
'''import seaborn as sns
from cf_matrix import make_confusion_matrix
labels =['True Neg','False Pos','False Neg','True Pos']
#ax =sns.heatmap(matrix, annot=True, fmt='d', group_names=labels)
make_confusion_matrix(matrix, group_names=labels, figsize=(8,6))
'''
#svm
kernal = ['linear','poly','rbf','sigmoid']
for k in kernal:
clf = SVC(kernel=k )
clf.fit(features_train,y_train)
y_pred =clf.predict(features_test)
#print('Kernal: ',k)
#tester(y_pred,y_test)
#neural net
model = keras.models.Sequential()
model.add(Dense(64, activation='relu', input_shape=(features_test.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
#from keras.layers import Sequential
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#print(model.metrics_names)
batch_size = 64
epochs = 3
history = model.fit(features_train, np.array(y_train), batch_size=batch_size,
epochs=epochs, verbose=1 )
score = model.evaluate(features_test, np.array(y_test), batch_size=batch_size,
verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
kernel = ['linear','poly','rbf','sigmoid']
models = [nb, clf,model]
models = {'Naive Bayes':nb, "Svm":clf, "Neural Network": model}
data = {'model_name': [], 'accuracy': [], "recall": [], 'precision': []
, "f1_score":[],'fbeta_score': [], 'ham_number': [], 'spam_number':[],
'true_neg':[], 'false_pos':[], 'false_neg':[], 'true_pos': []}
for names, model in models.items():
if names != 'Svm':
y_pred = model.predict(features_test)
if names == 'Neural Network':
y_final = []
for i in y_pred:
y_final.append(round(i[0]))
res = tester(y_final,y_test)
matrix = confusion_matrix(y_true=y_test,y_pred=y_final)
data['true_neg'].append(matrix[0][0])
data['false_pos'].append(matrix[0][1])
data['false_neg'].append(matrix[1][0])
data['true_pos'].append(matrix[1][1])
data['ham_number'].append(len(ham))
data['spam_number'].append(len(spam))
else:
res = tester(y_pred,y_test)
matrix = confusion_matrix(y_true=y_test,y_pred=y_pred)
data['true_neg'].append(matrix[0][0])
data['false_pos'].append(matrix[0][1])
data['false_neg'].append(matrix[1][0])
data['true_pos'].append(matrix[1][1])
data['ham_number'].append(len(ham))
data['spam_number'].append(len(spam))
count=0
for key, point in data.items():
if count <6:
if key == 'model_name':
data[key].append(names)
else:
try:
data[key].append(res[count])
count+=1
except:
pass
else:
for i in kernel:
clf = SVC(kernel=i)
clf.fit(features_train,y_train)
y_pred =clf.predict(features_test)
res = tester(y_pred,y_test)
matrix = confusion_matrix(y_true=y_test,y_pred=y_pred)
data['true_neg'].append(matrix[0][0])
data['false_pos'].append(matrix[0][1])
data['false_neg'].append(matrix[1][0])
data['true_pos'].append(matrix[1][1])
data['ham_number'].append(len(ham))
data['spam_number'].append(len(spam))
count = 0
for key, point in data.items():
if count <6:
if key == 'model_name':
data[key].append('Svm_'+i)
else:
try:
data[key].append(res[count])
count+=1
except:
pass
df = pd.DataFrame(data=data)
df_final = df_final.append(df)
print(q)
df_final['total_average'] = (df_final.iloc[:,1:6].sum(axis=1))/5
df_final['ham_spam_ratio'] = round(df_final['ham_number']/df_final['spam_number'] ,2)
df_final.to_csv('final_stats.csv')
import matplotlib.pyplot as plt
import plotly.express as px
df_model = df_final
fig = px.bar(data_frame=df_model, x='ham_spam_ratio', y='total_average',
color='model_name',color_discrete_sequence=px.colors.qualitative.D3, title ='Overall
Model Performance by Ratio')
'''fig.update_traces(marker=dict(size=11, opacity=.8,line=dict(width=2,
color='DarkSlateGrey')))'''
fig.update_layout(barmode='group')
fig.show()
df_model = df_final.groupby(by=['ham_spam_ratio'],
as_index=False)['total_average'].mean()
df_model.sort_values(by='ham_spam_ratio',ascending=True,inplace=True)
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
#df_model
fig = px.bar( data_frame=df_model, x='total_average', y='ham_spam_ratio',
color='ham_spam_ratio',color_discrete_sequence=px.colors.qualitative.D3, title='Ham
Spam Ratio Total Average Performance')
#fig.update_traces(marker=dict(size=11, opacity=.8,line=dict(width=2,
#color='DarkSlateGrey')))
fig.show()
#confusion
import numpy as np
data = np.asarray([[410,0],[3,211]])
from cf_matrix import make_confusion_matrix
#make_confusion_matrix(data)
labels =['True Neg','False Pos','False Neg','True Pos']
#ax =sns.heatmap(matrix, annot=True, fmt='d', group_names=labels)
make_confusion_matrix(data, group_names=labels, figsize=(8,6),title='SVM RBF
Kernel: Ham Spam Ratio 2:1')
#best overall performance
df_model = df_final.groupby(by=['model_name'], as_index=False)
df_model = df_model.apply(lambda x: x.sort_values('total_average',ascending=False))
df_model = df_model.groupby('model_name').head(1)
df_model.reset_index(drop=True, inplace=True)
df_model.to_csv('best_model_performance.csv')
#best precison
df_model = df_final.groupby(by=['model_name'], as_index=False)
df_model = df_model.apply(lambda x: x.sort_values('precision',ascending=False))
df_model = df_model.groupby('model_name').head(1)
df_model.reset_index(drop=True, inplace=True)
df_model.to_csv('best_model_precision.csv')
#best fbeta
df_model = df_final.groupby(by=['model_name'], as_index=False)
df_model = df_model.apply(lambda x: x.sort_values('precision',ascending=False))
df_model = df_model.groupby('model_name').head(1)
df_model.reset_index(drop=True, inplace=True)
df_model.to_csv('best_model_fbeta.csv')
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='fbeta_score',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='Fbeta Analysis for Each Model')
fig.show()
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='precision',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='Precision Analysis for Each
Model')
fig.show()
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='total_average',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='Total Average Analysis for Each
Model')
fig.show()
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='accuracy',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='Accuracy Analysis for Each
Model')
fig.show()
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='recall',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='Recall Analysis for Each Model')
fig.show()
df_model = df_final
df_model['ham_spam_ratio'] =df_model['ham_spam_ratio'].astype(str)
fig = px.bar(df_model, x='model_name', y='f1_score',
facet_col='ham_spam_ratio',facet_col_wrap=10,title='F1 Analysis for Each Model')
fig.show()